<a href="https://colab.research.google.com/github/ZakiKurdya/Predictive-Maintenance/blob/main/notebooks/deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science CSCI 3320 Final Project** | Predictive Maintenance Dataset ⚙️

---

© Zaki Kurdya

<a href="https://github.com/ZakiKurdya/Predictive-Maintenance">
<img src="https://cdn-icons-png.flaticon.com/512/25/25231.png" alt="github repository" width=35></a>

<a href="https://colab.research.google.com/drive/1DDEsCd9cCICfcxJQJMmKq9yyMYXAl3et?usp=sharing">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Google_Colaboratory_SVG_Logo.svg/1200px-Google_Colaboratory_SVG_Logo.svg.png" width=65>
</a>

# **Deployment** 📤

Demo the predictive machine learning model using [Gradio](https://gradio.app/)

In [1]:
!pip install gradio --quiet

Load the saved model and the dataframe

In [ ]:
!wget https://github.com/ZakiKurdya/Predictive-Maintenance/raw/main/saved-models/decision_tree_model.pkl

In [12]:
import pickle

loaded_model = pickle.load(open('/content/decision_tree_model.pkl', 'rb'))

In [13]:
import pandas as pd

dataset_url = 'https://raw.githubusercontent.com/motazsaad/Data-Science-CSCI-3320/main/project%20datasets/Predictive%20Maintenance/ai4i2020.csv'
dataset = pd.read_csv(dataset_url)

# rename dataset columns
dataset.rename(columns = {'Air temperature [K]':'Air temperature',
                     'Process temperature [K]':'Process temperature',
                     'Rotational speed [rpm]':'Rotational speed',
                     'Torque [Nm]':'Torque',
                     'Tool wear [min]':'Tool wear'},
          inplace = True)
dataset['Power'] = dataset[['Rotational speed', 'Torque']].product(axis=1)

In [14]:
import numpy as np

# normalize the feature using z-score normalization
def z_score(feature, value):
    mean = np.mean(dataset[feature])
    std = np.std(dataset[feature])
    return (float(value) - mean) / std

In [ ]:
import gradio as gr
from sklearn.preprocessing import StandardScaler

def predict(air_temperature, process_temperature, rotational_speed, torque, tool_wear, type):
    # normalize the inputs
    air_temperature = z_score('Air temperature', air_temperature)
    process_temperature = z_score('Process temperature', process_temperature)
    rotational_speed = z_score('Rotational speed', rotational_speed)
    torque = z_score('Torque', torque)
    tool_wear = z_score('Tool wear', tool_wear)
    power = z_score('Power', torque * rotational_speed)

    mapping = {'L': 0, 'M': 1, 'H': 2}
    temp = [{'Type':mapping[type], 'Air temperature':air_temperature,
             'Process temperature':process_temperature, 'Rotational speed':rotational_speed,
             'Torque':torque, 'Tool wear':tool_wear, 'Power':power}]

    input_data = pd.DataFrame(temp) 
    prediction = loaded_model.predict_proba(input_data)

    classes = ["No failure", "Machine failure"]
    prediction_dict = dict
    for record in prediction:
        prediction_dict = {classes[i] : record[i] for i in range(2)}
    
    maintenance = "No action required"
    if max(prediction_dict, key=prediction_dict.get) != "No failure":
        maintenance = "Need maintenance"

    return prediction_dict, maintenance

# create the user interface (inputs and outputs)
demo = gr.Interface(
    fn=predict,
    inputs=[gr.Slider(100, 350, label="Air temperature"),
            gr.Slider(100, 350, label="Process temperature"),
            gr.Number(label="Rotational speed"),
            gr.Number(label="Torque"),
            gr.Number(label="Tool wear"),
            gr.Radio(["L", "M", "H"], label="Type")],
    outputs=[gr.Label(num_top_classes=2, label="Result"), gr.components.Textbox(label="Action")]
)
demo.launch()